**UpVote if you find helpful**

In [1]:
!pip install -q autogluon

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.0/44.0 kB 1.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.2 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 1.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 259.5/259.5 kB 9.1 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 222.7/222.7 kB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 454.5/454.5 kB 16.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 382.4/382.4 kB 12.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 181.2/181.2 kB 7.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 69.1/69.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 275.8/275.8 kB 10.3 MB/s eta 0:00

In [2]:
# Import Library
import pandas as pd
import numpy as np
from autogluon.tabular import TabularPredictor


def add_feature_cross_terms(df, numerical_features):
    df_new = df.copy()
    for i in range(len(numerical_features)):
        for j in range(i + 1, len(numerical_features)):  
            feature1 = numerical_features[i]
            feature2 = numerical_features[j]
            cross_term_name = f"{feature1}_x_{feature2}"
            df_new[cross_term_name] = df_new[feature1] * df_new[feature2]
    return df_new




# Load data
train = pd.read_csv("/kaggle/input/playground-series-s5e5/train.csv")
test = pd.read_csv("/kaggle/input/playground-series-s5e5/test.csv")
submission = pd.read_csv("/kaggle/input/playground-series-s5e5/sample_submission.csv")



# Define numerical features
numerical_features = ["Age", "Height", "Weight", "Duration", "Heart_Rate", "Body_Temp"]



# Add cross terms to capture feature interactions
train = add_feature_cross_terms(train, numerical_features)
test = add_feature_cross_terms(test, numerical_features)




# Prepare training data with log-transformed target
train_data = train.copy()
train_data['log_Calories'] = np.log1p(train_data['Calories'])
train_data = train_data.drop(columns=['id', 'Calories'])




# Prepare test data, keeping 'id' for submission
test_data = test.copy()





# Initialize AutoGluon TabularPredictor
predictor = TabularPredictor(
    label='log_Calories',
    problem_type='regression',
    eval_metric='root_mean_squared_error',
    path='autogluon_model'
)



# Fit the model with bagging for robustness
predictor.fit(
    train_data,
    presets='best_quality',
    num_bag_folds=5,
    time_limit=7200  # Increase Time for better output
)

# Generate predictions on test data
log_preds = predictor.predict(test_data)

# Transform predictions back to original scale and clip
preds = np.expm1(log_preds)
preds = np.clip(preds, 1, 314)

# Submission file
submission['Calories'] = preds
submission.to_csv('submission.csv', index=False)

Verbosity: 2 (Standard Logging)
=================== System Info ===================
AutoGluon Version:  1.3.0
Python Version:     3.11.11
Operating System:   Linux
Platform Machine:   x86_64
Platform Version:   #1 SMP PREEMPT_DYNAMIC Sun Nov 10 10:07:59 UTC 2024
CPU Count:          4
Memory Avail:       29.83 GB / 31.35 GB (95.1%)
Disk Space Avail:   19.50 GB / 19.52 GB (99.9%)
Presets specified: ['best_quality']
Setting dynamic_stacking from 'auto' to True. Reason: Enable dynamic_stacking when use_bag_holdout is disabled. (use_bag_holdout=False)
Stack configuration (auto_stack=True): num_stack_levels=1, num_bag_folds=5, num_bag_sets=1
DyStack is enabled (dynamic_stacking=True). AutoGluon will try to determine whether the input data is affected by stacked overfitting and enable or disable stacking as a consequence.
	This is used to identify the optimal `num_stack_levels` value. Copies of AutoGluon will be fit on subsets of the data. Then holdout validation data is used to detect stacke

(_ray_fit pid=727) [1000]	valid_set's rmse: 0.0615723
(_ray_fit pid=727) [2000]	valid_set's rmse: 0.0611432 [repeated 4x across cluster] (Ray deduplicates logs by default. Set RAY_DEDUP_LOGS=0 to disable log deduplication, or see https://docs.ray.io/en/master/ray-observability/user-guides/configure-logging.html#log-deduplication for more options.)


(_ray_fit pid=726) 	Ran out of time, early stopping on iteration 1994. Best iteration is:
(_ray_fit pid=726) 	[1992]	valid_set's rmse: 0.0604478


(_ray_fit pid=916) [1000]	valid_set's rmse: 0.0627939 [repeated 2x across cluster]
(_ray_fit pid=916) [2000]	valid_set's rmse: 0.0622754
(_ray_fit pid=916) [3000]	valid_set's rmse: 0.0621232


(_ray_fit pid=916) 	Ran out of time, early stopping on iteration 3851. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=916) 	[3845]	valid_set's rmse: 0.0620573 [repeated 4x across cluster]
(_dystack pid=280) 	-0.0607	 = Validation score   (-root_mean_squared_error)
(_dystack pid=280) 	525.28s	 = Training   runtime
(_dystack pid=280) 	311.18s	 = Validation runtime
(_dystack pid=280) Fitting model: WeightedEnsemble_L2 ... Training model for up to 360.00s of the 555.96s of remaining time.
(_dystack pid=280) 	Ensemble Weights: {'LightGBMXT_BAG_L1': 0.95, 'KNeighborsDist_BAG_L1': 0.05}
(_dystack pid=280) 	-0.0606	 = Validation score   (-root_mean_squared_error)
(_dystack pid=280) 	0.61s	 = Training   runtime
(_dystack pid=280) 	0.01s	 = Validation runtime
(_dystack pid=280) Fitting 106 L2 models, fit_strategy="sequential" ...
(_dystack pid=280) Fitting model: LightGBMXT_BAG_L2 ... Training model for up to 555.28s of the 555.18s of remaining time.
(_dystack pid=280) 	Fitting 5 

(_ray_fit pid=1113) [1000]	valid_set's rmse: 0.0627265
(_ray_fit pid=1114) [1000]	valid_set's rmse: 0.0609608
(_ray_fit pid=1111) [1000]	valid_set's rmse: 0.0602214 [repeated 2x across cluster]


(_ray_fit pid=1114) 	Ran out of time, early stopping on iteration 1536. Best iteration is:
(_ray_fit pid=1114) 	[1532]	valid_set's rmse: 0.060649


(_ray_fit pid=1302) [1000]	valid_set's rmse: 0.0620647
(_ray_fit pid=1302) [2000]	valid_set's rmse: 0.0615058
(_ray_fit pid=1302) [3000]	valid_set's rmse: 0.0613061


(_ray_fit pid=1302) 	Ran out of time, early stopping on iteration 3037. Best iteration is: [repeated 4x across cluster]
(_ray_fit pid=1302) 	[3008]	valid_set's rmse: 0.061303 [repeated 4x across cluster]
(_dystack pid=280) 	-0.0609	 = Validation score   (-root_mean_squared_error)
(_dystack pid=280) 	514.74s	 = Training   runtime
(_dystack pid=280) 	303.8s	 = Validation runtime
(_dystack pid=280) Fitting model: WeightedEnsemble_L3 ... Training model for up to 360.00s of the -18.40s of remaining time.
(_dystack pid=280) 	Ensemble Weights: {'LightGBMXT_BAG_L1': 0.545, 'LightGBMXT_BAG_L2': 0.455}
(_dystack pid=280) 	-0.0602	 = Validation score   (-root_mean_squared_error)
(_dystack pid=280) 	0.67s	 = Training   runtime
(_dystack pid=280) 	0.02s	 = Validation runtime
(_dystack pid=280) AutoGluon training complete, total runtime = 1811.29s ... Best model: WeightedEnsemble_L3 | Estimated inference throughput: 180.3 rows/s (133334 batch size)
(_dystack pid=280) TabularPredictor saved. To load,

**Thank You !**